# Построение модели по с использованием ансамблевого обучения "Случайный лес" (Random Forest). Сохранение модели.

In [6]:
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
#from sklearn.externals import joblib  # Импортируем joblib для сохранения модели
    
# Шаг 1: Загружаем данные
file_path = 'D:\\new_fz_output_final.csv'
df = pd.read_csv(file_path)
    
# Убедимся, что в данных нет пропусков
df.dropna(subset=['Описание', 'Код_ОКПО-2'], inplace=True)
    
# Шаг 2: Разделяем данные на обучающую и тестовую выборки
X = df['Описание']
y = df['Код_ОКПО-2']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
# Шаг 3: Преобразуем текстовые данные в векторы
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)
    
# Шаг 4: Создаем и обучаем модель
model = RandomForestClassifier(n_estimators=100, random_state=42)
cv_scores = cross_val_score(model, X_train_vec, y_train, cv=5)
model.fit(X_train_vec, y_train)
    
# Шаг 5: Делаем предсказания и оцениваем модель
y_pred = model.predict(X_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
    
# Сохраняем модель и векторизатор
joblib.dump(model, 'random_forest_model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

Accuracy: 0.9005824965157417
Classification Report:
               precision    recall  f1-score   support

          41       0.91      0.65      0.76      9352
          42       0.92      0.92      0.92     24032
          43       0.86      0.92      0.89     32483
          71       0.95      0.96      0.96     18082

    accuracy                           0.90     83949
   macro avg       0.91      0.86      0.88     83949
weighted avg       0.90      0.90      0.90     83949



['vectorizer.pkl']

# Разрабатываем консольное приложение по определению кода ОКПО-2 по входному тексту.

In [2]:
import joblib  
import re  
import nltk  
from nltk.tokenize import word_tokenize  
from nltk.stem import SnowballStemmer  
from nltk.corpus import stopwords  
import os  

# Печатаем текущую директорию  
print(os.getcwd())  

# Загрузка необходимых ресурсов NLTK  
nltk.download('punkt')  
nltk.download('stopwords')  

# Инициализация стеммера и стоп-слов  
stemmer = SnowballStemmer('russian')  
stop_words = set(stopwords.words('russian'))  

class NLP:  
    def __init__(self, text):  
        self.text = text  

    def preprocess_text(self):  
        # Приведение к нижнему регистру и очистка текста от специальных символов  
        text = re.sub(r'[^а-яА-ЯёЁ\s]', '', self.text.lower())  
        # Токенизация и удаление стоп-слов, стемминг  
        processed_tokens = [stemmer.stem(token) for token in word_tokenize(text) if token not in stop_words]  
        return ' '.join(processed_tokens)  

def load_model_and_vectorizer():  
    model = joblib.load('random_forest_model.pkl')  
    vectorizer = joblib.load('vectorizer.pkl')  
    return model, vectorizer  

def classify_input(input_text, model, vectorizer):  
    input_vec = vectorizer.transform([input_text])  
    prediction = model.predict(input_vec)  
    return prediction[0]  

def main():  
    print("Добро пожаловать в классификатор")  
    model, vectorizer = load_model_and_vectorizer()  
    
    # Пример соответствия классов и групп ОКПО-2  
    class_to_okpo = {41, 42, 43, 71, 'Прочие'}  

    while True:  
        user_input = input("Введите предложение (или 'exit' для выхода): ")  
        if user_input.lower() == 'exit':  
            print("Выход из приложения.")  
            break  

        # Создание экземпляра NLP для предобработки текста  
        nlp = NLP(user_input)  
        processed_input = nlp.preprocess_text()  
        print("Обработанный текст:", processed_input)  
        
        # Классификация обработанного текста  
        prediction = classify_input(processed_input, model, vectorizer)  
        print("Предсказанная группа:", prediction)  

        # Проверяем нахождение предсказанного класса в нашем словаре  
        if prediction in class_to_okpo:  
            print(f"Код ОКПО-2 для этой группы: {prediction}")  
        else:  
            print("Ваше предложение относится к группе: 'Прочие'")  

if __name__ == "__main__":  
    main()

C:\Users\Димка


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Димка\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Димка\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Добро пожаловать в классификатор


Введите предложение (или 'exit' для выхода):  прочая закупка услуг (услуги по монтажу и пусконаладке вытяжных систем вентиляции в производственных помещениях раскройного участка швейного цеха)


Обработанный текст: проч закупк услуг услуг монтаж пусконаладк вытяжн сист вентиляц производствен помещен раскройн участк швейн цех
Предсказанный класс: 43
Код ОКПО-2 для этого класса: 43


...

Обработанный текст: монтаж демонтаж дорожн знак согласн гост р зам устаревш знак
Предсказанный класс: 42
Код ОКПО-2 для этого класса: 42


Введите предложение (или 'exit' для выхода):  На поставку специальных средств при нарушениях функций выделения  для обеспечения инвалидов  в 2022 году


Обработанный текст: поставк специальн средств нарушен функц выделен обеспечен инвалид год
Предсказанный класс: 71
Код ОКПО-2 для этого класса: 71


Введите предложение (или 'exit' для выхода):  exit


Выход из приложения.
